<a href="https://colab.research.google.com/github/antwandaher/opl/blob/master/Video_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Download Videos from YouTube

In [0]:
# !pip install pytube

In [0]:
import pytube

In [0]:
video_url='https://www.youtube.com/watch?v=jDC_DAXBG7M'

youtube=pytube.YouTube(video_url)
video=youtube.streams.first()
video.download(path_to_drive)

## Settings and Imports

In [39]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [0]:
from fastai.vision import *
from zipfile import ZipFile 
import pandas as pd
import numpy as np
import cv2

In [0]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline 

In [0]:
path_to_drive=Path('/gdrive/My Drive/MadridCohort/Project_damage')

In [43]:
os.getcwd()

'/content'

##Helper Functions 

In [0]:
# Relevant Librarires
from fastai.callbacks.hooks import *
from google.colab.patches import cv2_imshow
#!curl -o logo.png https://colab.research.google.com/img/colab_favicon_256px.png

In [0]:
## Function that generates de 7x7 heatmap array, y is the value of the category we want ot highlight (y=1 corresponds to positive damage in our case)

def hm_array(img,y):
  xb,_ = data.one_item(img)
  xb_im = Image(data.denorm(xb)[0])
  xb = xb.cuda()
  m=learn.model.eval()
  cat=y
  with hook_output(m[0]) as hook_a: 
    with hook_output(m[0], grad=True) as hook_g:
      preds = m(xb)
      preds[0,int(cat)].backward()
  acts = hook_a.stored[0].cpu()
  grad = hook_g.stored[0][0].cpu()

  grad_chan = grad.mean(1).mean(1)
  mult = (acts*grad_chan[...,None,None]).mean(0)

  return mult


# Calculating list of coordinates of activated pixels (points with values equal to 1)
def get_coords(img_path):
   img=open_image(img_path) #Opening image
   img_array_=hm_array(img,1)  #Creating array from image
   # Calculating widht and height of original image for re-scaling
   img_h=img.shape[1]
   img_w=img.shape[2]
   img_array = cv2.resize(img_array_.numpy(), (img_w, img_h))  # resize the Gradcam tensor
   # Calculating mean and standard deviation of heatmap which will be used to pick up activated part of image
   img_mean=np.mean(img_array)
   img_sd=np.std(img_array)
   # Picking up (1) most activated region of the heatmap. This could be further optimized
   img_array[img_array <=img_mean+img_sd]=0
   img_array[img_array > img_mean+img_sd]=1
   # Picking up (1) most activated region of the heatmap. This could be further optimized
   active_px = np.argwhere(img_array!=0)
   active_px = active_px[:,[1,0]]
   x,y,w,h = cv2.boundingRect(active_px)
   vertices=[(x,y),(x+w,y+h)]
   return vertices

In [0]:
## Drawing rectangle and text

def drawing_box(path_in,path_out,fname, color,label):
  '''

  path_in: string
  path_out: tring
  '''
  path_in=str(path_in)+'/'
  path_out=str(path_out)+'/'
  
  rgb = ()
  
  if 'red' in color:
    rgb=(0, 0, 255)
    pos=20
  elif 'blue' in color:
    rgb=(255, 0, 0)
    pos=40
  else:
    rgb=(0, 255, 0)
    pos=60
    
  img_path=path_in+fname #cv2 imread function requires path to be entered as a string
  img_test=cv2.imread(img_path)

  ## Saving coords

  coords_alex=get_coords(img_path)
  rect=cv2.rectangle(img_test,coords_alex[0],coords_alex[1],rgb, 5)
  rect=cv2.putText(img_test, label, (20, img_test.shape[0]-pos), cv2.FONT_HERSHEY_TRIPLEX, 2, rgb, 2)
  cv2.imwrite(path_out+'/'+fname,rect)

  return coords_alex


In [0]:
def save_heatmap(path_in,path_out,fname,y):
  path_in=str(path_in)+'/'
  path_out=str(path_out)+'/'
  img_path=path_in+fname
  img=open_image(img_path) #Opening image
  img_h=img.shape[1]
  img_w=img.shape[2]
  xb,_ = data.one_item(img)
  xb_im = Image(data.denorm(xb)[0])
  xb = xb.cuda()
  m=learn.model.eval()
  cat=y
  with hook_output(m[0]) as hook_a:
   with hook_output(m[0], grad=True) as hook_g:
     preds = m(xb)
     preds[0,int(cat)].backward()
  acts = hook_a.stored[0].cpu()
  grad = hook_g.stored[0][0].cpu()
  grad_chan = grad.mean(1).mean(1)
  hm = (acts*grad_chan[...,None,None]).mean(0)
  _,ax = plt.subplots()
  img.show(ax)
  ax.imshow(hm, alpha=0.6, extent=(0,img_w,img_h,0),
             interpolation='bilinear', cmap='magma');
  ## Removing white space around image
  plt.gca().set_axis_off()
  plt.subplots_adjust(top = 1, bottom = 0, right = 1, left = 0,
           hspace = 0, wspace = 0)
  plt.margins(0,0)
  plt.gca().xaxis.set_major_locator(plt.NullLocator())
  plt.gca().yaxis.set_major_locator(plt.NullLocator())
  plt.savefig(path_out+'/'+fname,bbox_inches = 'tight',
   pad_inches = 0)
  plt.close()
  #plt.show()


##Get Preds

In [0]:
def get_data_frames(learn, label_pos, label_neg, threshold=0.95):
  prob,cat = learn.get_preds(DatasetType.Test)
  preds_ = np.argmax(prob,1)
  preds__=[x.item() for x in preds_]
  label=[label_pos if x==1  else label_neg for x in preds__]
  idx=[x for x in learn.data.test_ds.items]
  final=[prob[x][preds__[x]].item() for x in range(len(preds__))]

  # We add all this info in a dataframe
  total=[]
  for i in range(len(preds__)):

    frame=int(str(idx[i])[27:-4])
    total.append([idx[i],final[i],label[i],frame])

  columns = ['name','prob','label','indx']
  total = pd.DataFrame(total, columns=columns)
  total.sort_values(by='indx',axis=0, inplace=True)
  
  total_threshold=total[total['prob']>threshold]
  total_threshold=total_threshold[total_threshold['label'].isin([label_pos])] 
  
  df_threshold=total_threshold.copy()
  df_threshold.drop(labels=['prob','indx'],axis=1,inplace=True)
  df_threshold['name']=[str(x)[1:]for x in df_threshold['name']]
  df_threshold.head()
  return total_threshold, df_threshold


In [0]:
def get_data_bunch(df_threshold):
  return (ImageList.from_df(path='/',df=df_threshold)
        .split_none()
        .label_empty()
        .transform(tfms=[[],[]],size=(299,299))
        .databunch(bs=32)).normalize(imagenet_stats)

In [0]:
def draw_on_learner(total_threshold, box_color, label_pos):
  for index, row in total_threshold.iterrows():
      fname=str(row['name'])
      fname=fname[22:]
      drawing_box(p_video,p_video_out,fname, box_color, label_pos)
      #save_heatmap(p_video,p_video_out,fname,1) ## last sets the label of the category we want to activate

##Extract Video Frames

In [0]:
# Code for reset
#!rm -rf damage/video
!rm -rf damage/video_output

In [51]:
# # Input and output setup
p_video = Path('/content/damage/video/')
p_video_out=Path('/content/damage/video_output/')

# p_video.mkdir()
# p_video_out.mkdir()
    
!mkdir damage
!mkdir damage/video
!mkdir damage/video_output


mkdir: cannot create directory ‘damage’: File exists
mkdir: cannot create directory ‘damage/video’: File exists
mkdir: cannot create directory ‘damage/video_output’: File exists


In [0]:
videoFile=  path_to_drive/'Video_challenge/box_madrid_cohort.mp4'

cap = cv2.VideoCapture(str(videoFile))   # capturing the video from the given path
frameRate = 3
count = 0

while(cap.isOpened()):
    frameId = cap.get(1) #current frame number
    ret, frame = cap.read()
    if (ret != True):
        break
    if (frameId % math.floor(frameRate) == 0):
        filename ="frame%d.png" % count;count+=1
        cv2.imwrite(str(p_video/filename), frame)
cap.release()

In [16]:
output_count = sum('.png' in x for x in os.listdir(p_video))    # list(filter(lambda x: '.jpg' in x, os.listdir(p_video))).length
print ("Done! Output .png count: {} files".format(output_count))

Done! Output .png count: 914 files


In [0]:
!cp -a /content/damage/video/. /content/damage/video_output/

##Load Video Dataframe

In [0]:
 learner_list = [
#      {'folder': 'model', 'name':'RN34_Callback.pkl', 'label':'Structure_Damage', 'label_pos': 'positive', 'label_neg': 'negative', 'fp_16': False, 'box-color':'red'},
#      {'folder': 'models_for_video', 'name':'platform-demo4_week4_models_Water-damage.pkl', 'label':'Water_Damage', 'label_pos': 'water_d', 'label_neg': 'no_water_d', 'fp_16': False, 'box-color':'blue'},
     {'folder': 'models_for_video', 'name':'overgrown_landscape_SF.pkl', 'label':'Overgrown_Landscape', 'label_pos': 'overgrown', 'label_neg': 'undergrown', 'fp_16': True, 'box-color':'green'},
     {'folder': 'models_for_video', 'name':'platform-demo4_week4_models_dead_landscape.pkl', 'label':'Dead_Landscape', 'label_pos': 'dead', 'label_neg': 'healthy', 'fp_16': False, 'box-color':'green'}
  ]
#     
# learn = load_learner(path_to_drive/'model/','RN34_Callback.pkl',test=ImageList.from_folder(p_video))
# learn2 = load_learner(path_to_drive/'model/','RN34_Callback.pkl',test=ImageList.from_folder(p_video))

#learner_list = [{'folder': 'models_for_video', 'name':'platform-demo4_week4_models_dead_landscape.pkl', 'label':'Dead_Landscape', 'label_pos': 'dead', 'label_neg': 'healthy', 'fp_16': False, 'box-color':'green'}]

In [55]:
model_stack, success_count = [], []
for model_info in learner_list:
  
  if not model_info['fp_16']:
    learn = load_learner(path_to_drive/model_info['folder'],model_info['name'],test=ImageList.from_folder(p_video))
  else:
    learn = load_learner(path_to_drive/model_info['folder'],model_info['name'],test=ImageList.from_folder(p_video)).to_fp32()
  total_threshold, df_threshold = get_data_frames(learn, model_info['label_pos'], model_info['label_neg'])
  print(model_info['name'], df_threshold.name.count())
  data = get_data_bunch(df_threshold)
  draw_on_learner(total_threshold, model_info['box-color'], model_info['label'])
  model_stack.append(model_info['name'])
  success_count.append(df_threshold.name.count())
  learn.destroy()
# 
for i in range(len(model_stack)):
  print('Printed on {} images using {}'.format(success_count[i],model_stack[i]))

RuntimeError: ignored

## Output Video

In [0]:
import cv2
import os

image_folder = str(p_video_out)+'/'
video_name = 'full_video_0_95.avi'

images = [img for img in os.listdir(image_folder) if img.endswith(".png")]
frame = cv2.imread(os.path.join(image_folder, images[0]))
heigth, width, layers = frame.shape
#heigth, width, layers = int(height/2), int(width/2), layers
ratio=width/heigth
width=600
heigth=int(width/ratio)

#(width,height)=img.shape[1],img.shape[0]
frs=25/frameRate
video = cv2.VideoWriter(video_name, 0, frs,(width,heigth))

for i in range(0,len(images)):
    fname ="frame%d.png" % i;

    full_fname=(image_folder+fname)

    image=(cv2.imread(full_fname))
    resize = cv2.resize(image,(width, heigth), interpolation = cv2.INTER_LINEAR)
    video.write(resize)


#for image in images:
#    video.write(cv2.imread(os.path.join(image_folder, image)))

cv2.destroyAllWindows()
video.release()

In [0]:
!zip -r /content/full_video.zip /content/full_video_0_95_hm.avi

  adding: content/full_video_0_95_hm.avi (deflated 59%)
